In [14]:
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
import pandas as pd

# Prepare data

Ta sẽ sử dụng các feature vector được trích xuất bằng Openface (Xem kĩ readme step_by_step_face_recognition để biết thêm thông tin)

In [4]:
label_list = os.listdir("/Users/trananhvu/Documents/CV/data/sample")
label_list.remove(".DS_Store")
label_list.remove("Unknown")
label_list

['Robert Downey Jr',
 'Chris Evans',
 'Mark Ruffalo',
 'Chris Hemsworth',
 'Scarlett Johansson',
 'Tom Holland']

In [5]:
feature_path = "/Users/trananhvu/Documents/CV/data/feature/hog_openface"
count = 0
label_idx = []
feature = []
idx2label = {}
for i in os.listdir(feature_path):
    if i in label_list:
        idx2label[count]=i
        for j in os.listdir(os.path.join(feature_path, i)):
            feat=np.load(os.path.join(os.path.join(feature_path, i), j))
            feature.append(feat.reshape(128).tolist()) 
            label_idx.append(count)
        count+=1
    else:
        continue


# PCA and plot

In [6]:
pca = PCA(n_components=3)
feature_pca = pca.fit_transform(feature)

In [7]:
feature_pca.shape

(1105, 3)

In [23]:
coord_info = {'x':[], 'y':[], 'z':[], 'label':[]}
for feat, idx in zip(feature_pca, label_idx):
    coord_info['x'].append(feat[0])
    coord_info['y'].append(feat[1])
    coord_info['z'].append(feat[2])
    coord_info['label'].append(idx2label[idx])
info = pd.DataFrame.from_dict(coord_info)
fig = px.scatter_3d(info, x='x', y='y', z='z',
              color='label')
fig.show()

# Clustering

In [10]:
km = KMeans(
    n_clusters=6, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)

predict_label_idx = km.fit_predict(feature)

In [11]:
clustering_result = pd.DataFrame({"predict": predict_label_idx, "ground_truth": label_idx})
clustering_result

,predict,ground_truth
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
1100,3,5
1101,3,5
1102,3,5
1103,3,5


In [12]:
clustering_result.to_csv("/Users/trananhvu/Documents/CV/CV_internship/face_recognition/cosine_distance_face_recognition/sample/clustering_result.csv", index=False)

Ta nhận thấy sử dụng Kmean cho phân cụm khuôn mặt làm khá là tốt, chứng tỏ các bộ trích xuất đặc trưng khuôn mặt hoạt động khá ổn